In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec
import pandas as pd

In [3]:
df = pd.read_csv(r'E:\importante\Master\Procesamiento lenguaje natural\tareas\TP1\NLP-text-vectorization\2-gensim\bruno-mars.txt', sep='/n', header=None)
df.head()

C:\Users\laum_\AppData\Local\Temp\ipykernel_7828\2917244481.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(r'E:\importante\Master\Procesamiento lenguaje natural\tareas\TP1\NLP-text-vectorization\2-gensim\bruno-mars.txt', sep='/n', header=None)


,0
0,Now greetings to the world! Standing at this l...
1,"Whiskey coming through my pores,"
2,Feeling like I run this whole block.
3,Lotto tickets cheap beer
4,"That's why you can catch me here,"


In [4]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 3270


In [5]:
import tensorflow as tf
print(tf.__version__)


2.15.0


In [6]:
from keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [7]:
# Demos un vistazo
sentence_tokens[:2]

[['now',
  'greetings',
  'to',
  'the',
  'world',
  'standing',
  'at',
  'this',
  'liquor',
  'store'],
 ['whiskey', 'coming', 'through', 'my', 'pores']]

In [8]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [9]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=300,       # dimensionalidad de los vectores 
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [10]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [11]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=20,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 221229.265625
Loss after epoch 1: 143823.140625
Loss after epoch 2: 141473.21875
Loss after epoch 3: 145537.5
Loss after epoch 4: 141571.0625
Loss after epoch 5: 136788.8125
Loss after epoch 6: 128076.0
Loss after epoch 7: 110052.25
Loss after epoch 8: 106101.75
Loss after epoch 9: 104805.375
Loss after epoch 10: 101826.25
Loss after epoch 11: 99709.875
Loss after epoch 12: 98406.375
Loss after epoch 13: 96169.375
Loss after epoch 14: 95177.25
Loss after epoch 15: 94200.0
Loss after epoch 16: 93035.625
Loss after epoch 17: 88440.875
Loss after epoch 18: 84515.75
Loss after epoch 19: 83007.75


(328281, 538000)

In [12]:
w2v_model.wv.most_similar(positive=["love"], topn=10)

[('gorillas', 0.6890413165092468),
 ('madly', 0.6688576340675354),
 ('faith', 0.6490115523338318),
 ('promise', 0.6410069465637207),
 ('makes', 0.6313118934631348),
 ('yes', 0.6311607956886292),
 ('had', 0.6278324723243713),
 ('making', 0.6214283108711243),
 ('things', 0.6207209825515747),
 ('wanted', 0.6192373037338257)]

In [13]:
w2v_model.wv.most_similar(positive=["magic"], topn=10)

[('karat', 0.9777292013168335),
 ('four', 0.9299569725990295),
 ('twenty', 0.913456916809082),
 ('air', 0.8786908984184265),
 ('nude', 0.7945184707641602),
 ('ohhh', 0.7635891437530518),
 ('strawberry', 0.7512890100479126),
 ('singing', 0.7327882051467896),
 ('voices', 0.7247868776321411),
 ('toe', 0.7244638204574585)]

In [14]:
w2v_model.wv.most_similar(positive=["money"], topn=10)

[('own', 0.8130840063095093),
 ('smile', 0.7870734333992004),
 ('music', 0.7815853953361511),
 ('dance', 0.7321241497993469),
 ('might', 0.7285992503166199),
 ('gimme', 0.7221306562423706),
 ('mess', 0.6949920058250427),
 ('sure', 0.6894083619117737),
 ('make', 0.6875020861625671),
 ('car', 0.6837464570999146)]

In [15]:
# el método `get_vector` permite obtener los vectores:
vector_love = w2v_model.wv.get_vector("love")
print(vector_love)

[ 0.33832002 -0.02793505  0.29605755  0.10643134  0.11928408  0.16385105
  0.37209952  0.01606201 -0.03164533 -0.06848458  0.16155653  0.05835926
 -0.17533244  0.0295419  -0.10929884  0.38732535  0.03078085 -0.07089821
 -0.17165694 -0.18333575 -0.17624669  0.0379191  -0.06422986  0.07858153
 -0.18162373 -0.07019068 -0.03589644 -0.22576256 -0.06516281  0.10216952
  0.20046917 -0.43042082  0.15854418  0.15450338  0.2770302   0.01694411
 -0.03515081 -0.0891881  -0.11370579 -0.04127058 -0.01392182 -0.40231317
  0.13243765  0.24570216 -0.04556881  0.05075917  0.06710969 -0.28652456
 -0.42219287  0.15872    -0.05366286 -0.00973671 -0.0092794   0.25798017
 -0.158353    0.16494407 -0.20379779 -0.13509768  0.05504715  0.10340629
 -0.2920578   0.29990807 -0.00836211  0.10011449 -0.1760401  -0.05962789
 -0.02685345  0.16981716  0.27211127 -0.19739908  0.01437878 -0.10890414
  0.11357526 -0.11681451  0.0493796   0.04619116  0.16499355 -0.05346693
 -0.06064418  0.42722815 -0.01494612 -0.6051871  -0

In [16]:
# el método `most_similar` también permite comparar a partir de vectores
w2v_model.wv.most_similar(vector_love)

[('love', 1.0),
 ('gorillas', 0.6890413165092468),
 ('madly', 0.6688576340675354),
 ('faith', 0.6490115523338318),
 ('promise', 0.6410068869590759),
 ('makes', 0.6313118934631348),
 ('yes', 0.6311607956886292),
 ('had', 0.6278324127197266),
 ('making', 0.6214283108711243),
 ('things', 0.6207209825515747)]

Hay relaciones fuertes que estan formadas por canciones particules, la cancion "gorilla" de bruno mars es la culpable de que love esta tan relacionada a la palabra "gorillas", otras relaciones tiene un mayor sentido en terminons generales, como "madly", "making" y "faith". Con esto podemos ver como si bien los resultados nos sirven parara construir relaciones en general, se ven altamente afectados por inclinaciones particulares de los datos de entrenamiento.

In [17]:
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np                                  

def reduce_dimensions(model, num_dimensions = 2 ):
     
    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [18]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show() # esto para plotly en colab

Algunas relaciones mostradas son esperables como "my", "your" y "body" alrededor de Y 15 ,X -7. Tambien se puede destacar como palabras en español estan todas agrupadas en un conojunto separadas del resto.

In [20]:
# Graficar los embedddings en 3D

vecs, labels = reduce_dimensions(w2v_model,3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
fig.show() # esto para plotly en colab

In [21]:
# También se pueden guardar los vectores y labels como tsv para graficar en
# http://projector.tensorflow.org/


vectors = np.asarray(w2v_model.wv.vectors)
labels = list(w2v_model.wv.index_to_key)

np.savetxt("vectors.tsv", vectors, delimiter="\t")

with open("labels.tsv", "w") as fp:
    for item in labels:
        fp.write("%s\n" % item)

Como conclusión podemos ver que la utilidad de usar canciones u otro tipo de textos que contenga secciones repetidas, como los estribillos, esta limitado por las fuertes relaciones que generan en palabras que no estan relacionadas en otros contextos.